#### 2013 and 2018 parking inventory comparison
##### Author: Polina Butrina 

From Carol's email: "it could be worth doing some preliminary analysis at a lower level (tract maybe?) to see if there are significant geographic differences within any given study area. But I’m also OK with keeping the analysis at the study area summary level, especially given the big methodological shifts between the two datasets. "

", it would be helpful to run a simple comparison between the 2013 and 2018 results, identifying where the biggest differences emerge (per the Trend article). To the degree that we can, identify where these differences are attributable to real changes on the ground vs. data collection methodology—a conversation with Peter could be helpful on this front. A longer time trend analysis that includes additional historical data points might also be helpful for identifying methodology-based anomalies as well."


In [358]:
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [359]:
#set working directory
path = "C:\\Users\\pbutrina\\Documents\\Python Scripts\\parking_inventory"
os.chdir(path)
os.getcwd()

'C:\\Users\\pbutrina\\Documents\\Python Scripts\\parking_inventory'

In [360]:
# download 2013 and 2018 parking data
parking_cap_zones_2018 = pd.read_excel('parking_summaries_2018.xlsx', sheet_name='CapacityZones', index = False)
parking_occup_zones_2013 = pd.read_excel('copy_of_parking_summaries_13.xlsx', sheet_name='OccupancyZones', index = False)


In [361]:
data_2018 = parking_cap_zones_2018
data_2013 = parking_occup_zones_2013

In [5]:
data_2013.head()

,County,City,Zone,Tract,Block,Number of Lots,Total Stalls,Total AM Car Count,Total PM Car Count,Average Total Stalls,Average AM Car Count,Average PM Car Count,AM Occupancy Rate,PM Occupancy Rate,Average Daily Occupancy Rate
0,King,Bellevue,1,23803,1000,3.0,194.0,73,86,65,24,29,0.369231,0.446154,0.407692
1,King,Bellevue,1,23803,1001,6.0,387.0,182,225,65,30,38,0.461538,0.584615,0.523077
2,King,Bellevue,1,23803,1002,10.0,263.0,84,130,26,8,13,0.307692,0.5,0.403846
3,King,Bellevue,1,23803,1003,3.0,225.0,129,129,75,43,43,0.573333,0.573333,0.573333
4,King,Bellevue,1,23803,1004,6.0,2078.0,1019,1080,346,170,180,0.491329,0.520231,0.50578


In [6]:
parking_occup_zones_2013.describe()

,Number of Lots,Total Stalls
count,981.000000,981.000000
mean,8.296636,631.181448
std,86.115565,6590.145362
min,1.000000,0.000000
25%,1.000000,31.000000
50%,2.000000,79.000000
75%,3.000000,194.000000
max,2443.000000,180676.000000


In [24]:
parking_cap_zones_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987 entries, 0 to 986
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   County                    979 non-null    float64
 1   City                      986 non-null    object 
 2   Zone                      970 non-null    float64
 3   Tract                     970 non-null    float64
 4   Block                     970 non-null    float64
 5   Number of Lots            986 non-null    float64
 6   Total Stalls              986 non-null    float64
 7   Average Number of Stalls  986 non-null    float64
dtypes: float64(7), object(1)
memory usage: 61.8+ KB


In [62]:
data_2013.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 973 entries, 0 to 980
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   County                        972 non-null    object 
 1   City                          972 non-null    object 
 2   Zone                          972 non-null    float64
 3   Tract                         972 non-null    float64
 4   Block                         972 non-null    float64
 5   Number of Lots                972 non-null    float64
 6   Total Stalls                  972 non-null    float64
 7   Total AM Car Count            972 non-null    object 
 8   Total PM Car Count            972 non-null    object 
 9   Average Total Stalls          972 non-null    object 
 10  Average AM Car Count          972 non-null    object 
 11  Average PM Car Count          972 non-null    object 
 12  AM Occupancy Rate             972 non-null    object 
 13  PM Oc

Check if the number of zones, tracts, and blocks is the same in 2013 and 2018; Analysis shows that there were surveyed 44 tracts in 2013 and 42 in 2018; 24 cities in 2013 and 25 cities in 2018. See the detailed tables below

In [91]:
#unique number of zones, tracts, and blocks surveyed in 2013 and 2018
data_2013.nunique()

County                            4
City                             15
Zone                             20
Tract                            43
Block                           254
Number of Lots                   19
Total Stalls                    371
Total AM Car Count              270
Total PM Car Count              271
Average Total Stalls            264
Average AM Car Count            190
Average PM Car Count            200
AM Occupancy Rate               511
PM Occupancy Rate               514
Average Daily Occupancy Rate    644
Tract_Parking                    43
dtype: int64

In [92]:
data_2018.nunique()

County                        4
City                         25
Zone                         20
Tract                        42
Block                       256
Number of Lots               27
Total Stalls                390
Average Number of Stalls    290
Tract_Parking                42
dtype: int64

checking the change in parking in tracts between 2013 and 2018

In [364]:
#first, need to convert strings from data_2013 to numerical data to be able to calculate the difference between 2013 and 2018 data
#need to remove "-" in the columns
data_2013 = data_2013[data_2013.Zone != "-"]
data_2013[["Zone", "Tract","Block"]] = data_2013[["Zone", "Tract","Block"]].apply(pd.to_numeric)
data_2018[["Zone", "Tract","Block"]] = data_2018[["Zone", "Tract","Block"]].apply(pd.to_numeric)

In [365]:
#summing  together parking by tracts for 2013 and 2018
data_2013_total_parking = data_2013.groupby("Tract").sum()[["Total Stalls"]]
data_2018_total_parking = data_2018.groupby("Tract").sum()[["Total Stalls"]]

In [366]:
#renaming the new column
data_2013_total_parking = data_2013_total_parking.rename(columns={"Total Stalls": "Total Stalls 2013"})
data_2018_total_parking = data_2018_total_parking.rename(columns={"Total Stalls": "Total Stalls 2018"})

In [367]:
#creating a new df with 2013 and 2018 parking stalls grouped by the tract by merging two tables
parking_dif = pd.merge(data_2013_total_parking, data_2018_total_parking, how='outer', on='Tract')

In [368]:
#calculating absolute difference and % difference
parking_dif['abs_dif'] = parking_dif['Total Stalls 2018'] - parking_dif['Total Stalls 2013']
parking_dif['perc_dif'] = (parking_dif['Total Stalls 2018'] - parking_dif['Total Stalls 2013'])/parking_dif['Total Stalls 2013']*100
parking_dif = parking_dif.dropna()

In [14]:
parking_dif.sort_values(by = ['perc_dif','abs_dif'], ascending=False)

,Total Stalls 2013,Total Stalls 2018,abs_dif,perc_dif
Tract,,,,
5302.0,2820.0,10703.0,7883.0,279.539007
81100.0,1317.0,4512.0,3195.0,242.596811
90900.0,968.0,1985.0,1017.0,105.061983
90102.0,773.0,1472.0,699.0,90.426908
9300.0,4538.0,6050.0,1512.0,33.318643
7200.0,11127.0,14112.0,2985.0,26.826638
8200.0,9039.0,10877.0,1838.0,20.334108
23700.0,2145.0,2357.0,212.0,9.883450
23804.0,29551.0,32342.0,2791.0,9.444689


In [369]:
#calculating total % parking difference 2013 vs 2018 
(parking_dif['Total Stalls 2018'].sum()-parking_dif['Total Stalls 2013'].sum())/parking_dif['Total Stalls 2013'].sum()*100

12.789745179215833

In [190]:
#number of the tracts that we would like to analyze
tract = 81100

In [133]:
#checking what blocks in 2013 were included in 2018 and vice versa
a = data_2013[data_2013.Tract== tract].Block.unique()
b = data_2018[data_2018.Tract== tract].Block.unique()

#how many tracts were both in 2013 and 2018
len(set(a) & set(b))

4

In [132]:
print (len(set(a)),len(set(b)))


5 4


In [137]:
set(b) - set(a)

set()

In [44]:
len(set(b))

16

In [139]:
#checking how many parking is in new blocks surveyed
data_2018[(data_2018.Block.isin(set(b) - set(a))) & (data_2018.Tract == tract)]['Total Stalls'].sum()

0.0

In [138]:
data_2018[(data_2018.Block.isin(set(b) - set(a))) & (data_2018.Tract == tract)]

,County,City,Zone,Tract,Block,Number of Lots,Total Stalls,Average Number of Stalls


In [140]:
data_2013[(data_2013.Block.isin(set(a) - set(b))) & (data_2013.Tract == tract)]

,County,City,Zone,Tract,Block,Number of Lots,Total Stalls,Total AM Car Count,Total PM Car Count,Average Total Stalls,Average AM Car Count,Average PM Car Count,AM Occupancy Rate,PM Occupancy Rate,Average Daily Occupancy Rate
57,King,Bellevue,6.0,23801.0,1003.0,5.0,59.0,33,33,12,7,7,0.583333,0.583333,0.583333


In [191]:
park_dif_block_merged = pd.merge(data_2013[data_2013.Tract== tract], data_2018[data_2018.Tract== tract], how='outer', on='Block').dropna()
park_dif_block = park_dif_block_merged[['Tract_x','Block', 'Total Stalls_x', 'Total Stalls_y' ]]
park_dif_block = park_dif_block.rename(columns={"Total Stalls_x": "Total Stalls 2013","Total Stalls_y": "Total Stalls 2018"})
park_dif_block['abs_dif'] = park_dif_block['Total Stalls 2018'] - park_dif_block['Total Stalls 2013']
park_dif_block['perc_dif'] = (park_dif_block['Total Stalls 2018'] - park_dif_block['Total Stalls 2013'])/park_dif_block['Total Stalls 2013']*100

In [192]:
park_dif_block_merged.head()

,County_x,City_x,Zone_x,Tract_x,Block,Number of Lots_x,Total Stalls_x,Total AM Car Count,Total PM Car Count,Average Total Stalls,...,AM Occupancy Rate,PM Occupancy Rate,Average Daily Occupancy Rate,County_y,City_y,Zone_y,Tract_y,Number of Lots_y,Total Stalls_y,Average Number of Stalls
0,Kitsap,Bremerton,7.0,81100.0,1002.0,1.0,189.0,40,42,189,...,0.21164,0.222222,0.216931,35.0,Bremerton,7.0,81100.0,1.0,189.0,189.0
1,Kitsap,Bremerton,7.0,81100.0,1003.0,5.0,90.0,27,29,18,...,0.277778,0.333333,0.305556,35.0,Bremerton,7.0,81100.0,5.0,90.0,18.0
2,Kitsap,Bremerton,7.0,81100.0,1004.0,2.0,28.0,10,12,14,...,0.357143,0.428571,0.392857,35.0,Bremerton,7.0,81100.0,2.0,28.0,14.0
3,Kitsap,Bremerton,7.0,81100.0,1005.0,1.0,8.0,3,3,8,...,0.375,0.375,0.375,35.0,Bremerton,7.0,81100.0,1.0,8.0,8.0
4,Kitsap,Bremerton,7.0,81100.0,1006.0,2.0,30.0,11,10,15,...,0.4,0.333333,0.366667,35.0,Bremerton,7.0,81100.0,2.0,30.0,15.0


In [193]:
park_dif_block.sort_values(by=['perc_dif','abs_dif'], ascending=False).query("perc_dif != 0")

,Tract_x,Block,Total Stalls 2013,Total Stalls 2018,abs_dif,perc_dif
17,81100.0,2016.0,127.0,72.0,-55.0,-43.307087


In [131]:
#next block
data_2018[data_2018.Tract== tract]

,County,City,Zone,Tract,Block,Number of Lots,Total Stalls,Average Number of Stalls
48,33.0,Bellevue,5.0,23801.0,1013.0,3.0,155.0,52.0
53,33.0,Bellevue,6.0,23801.0,1002.0,1.0,288.0,288.0
54,33.0,Bellevue,6.0,23801.0,1004.0,2.0,20.0,10.0
55,33.0,Bellevue,6.0,23801.0,1005.0,3.0,93.0,31.0


In [63]:
data_2013[data_2013.Tract== tract]

,County,City,Zone,Tract,Block,Number of Lots,Total Stalls,Total AM Car Count,Total PM Car Count,Average Total Stalls,Average AM Car Count,Average PM Car Count,AM Occupancy Rate,PM Occupancy Rate,Average Daily Occupancy Rate
223,King,Seattle,11.0,7200.0,2006.0,1.0,19.0,15,17,19,15,17,0.789474,0.894737,0.842105
224,King,Seattle,11.0,7200.0,2007.0,1.0,45.0,38,29,45,38,29,0.844444,0.644444,0.744444
225,King,Seattle,11.0,7200.0,2008.0,1.0,205.0,151,147,205,151,147,0.736585,0.717073,0.726829
226,King,Seattle,11.0,7200.0,2009.0,1.0,29.0,21,24,29,21,24,0.724138,0.827586,0.775862
227,King,Seattle,11.0,7200.0,2010.0,1.0,7.0,4,3,7,4,3,0.571429,0.428571,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,King,Seattle,18.0,7200.0,1074.0,1.0,28.0,6,5,28,6,5,0.214286,0.178571,0.196429
424,King,Seattle,18.0,7200.0,1075.0,2.0,257.0,93,82,129,47,41,0.364341,0.317829,0.341085
425,King,Seattle,18.0,7200.0,1076.0,4.0,184.0,80,60,46,20,15,0.434783,0.326087,0.380435
426,King,Seattle,18.0,7200.0,1078.0,6.0,225.0,76,76,38,13,13,0.342105,0.342105,0.342105


In [ ]:
# Find the parking difference for the tracts AND blocks that were surveyed in both 2013 and 2018

In [178]:
park_dif_trblk_merged = pd.merge(data_2013, data_2018, how='inner', left_on=['Tract','Block','Zone'], right_on = ['Tract','Block','Zone']).dropna()

In [179]:
park_dif_trblk_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 911 entries, 0 to 910
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   County_x                      911 non-null    object 
 1   City_x                        911 non-null    object 
 2   Zone                          911 non-null    float64
 3   Tract                         911 non-null    float64
 4   Block                         911 non-null    float64
 5   Number of Lots_x              911 non-null    float64
 6   Total Stalls_x                911 non-null    float64
 7   Total AM Car Count            911 non-null    object 
 8   Total PM Car Count            911 non-null    object 
 9   Average Total Stalls          911 non-null    object 
 10  Average AM Car Count          911 non-null    object 
 11  Average PM Car Count          911 non-null    object 
 12  AM Occupancy Rate             911 non-null    object 
 13  PM Oc

In [180]:
park_dif_trblk_merged = park_dif_trblk_merged[['Tract','Block', 'Total Stalls_x', 'Total Stalls_y' ]]
park_dif_trblk_merged = park_dif_trblk_merged.rename(columns={"Total Stalls_x": "Total Stalls 2013","Total Stalls_y": "Total Stalls 2018"})

In [181]:
(park_dif_trblk_merged['Total Stalls 2018'].sum()-park_dif_trblk_merged['Total Stalls 2013'].sum())/park_dif_trblk_merged['Total Stalls 2013'].sum()*100

5.823942176696251

In [170]:
tr_bl_2013 = set(zip(data_2013.Tract, data_2013.Block,data_2013.Zone))
tr_bl_2018 = set(zip(data_2018.Tract, data_2018.Block,data_2018.Zone))

In [169]:
print(len(tr_bl_2013), len(tr_bl_2018))

973 987


In [167]:
print(len(tr_bl_2013), len(tr_bl_2018))

968 987


In [171]:
len(tr_bl_2013 & tr_bl_2018)

911

In [ ]:
#checking for duplicates

In [182]:
df = data_2018[data_2018.duplicated(subset=['Tract','Block'], keep=False)]
print (df)

     County                       City  Zone    Tract   Block  Number of Lots  \
64     33.0             Bellevue Total   NaN      NaN     NaN           242.0   
148    33.0                    Seattle   6.0   8100.0  3010.0             3.0   
157    33.0                    Seattle   7.0   8100.0  3010.0             1.0   
443     NaN                        CBD   NaN      NaN     NaN            88.0   
444     NaN                 First Hill   NaN      NaN     NaN            86.0   
445     NaN     International District   NaN      NaN     NaN            96.0   
446     NaN           Lower Queen Anne   NaN      NaN     NaN           254.0   
447     NaN                    Regrade   NaN      NaN     NaN           141.0   
448     NaN                 Waterfront   NaN      NaN     NaN            46.0   
449    33.0              Seattle Total   NaN      NaN     NaN           711.0   
544    33.0  University District Total   NaN      NaN     NaN           246.0   
553    35.0           Bainbr

In [ ]:
# Prep data for the Tableau
#Need to transform tract and blocks to match census shape files (need to add leading zeros)

In [199]:
#cols = ['Tract', 'Block'] 
#data_2013[cols] = data_2013[cols].dropna().apply(np.int64) 
#data_2018[cols] = data_2018[cols].dropna().applymap(np.int64)


3

In [370]:
cols = ['Tract', 'Block'] 
data_2013[cols] = data_2013[cols].dropna() 
data_2018[cols] = data_2018[cols].dropna() 
for col in cols:   
    data_2013[col] = data_2013[col].apply(lambda x: int(x) if x == x else "")

In [371]:
for col in cols:   
    data_2018[col] = data_2018[col].apply(lambda x: int(x) if x == x else "")

In [ ]:
#Create a table with all matched blocks and tracts in 2013 and 2018 (using th


In [386]:
#Creating a subset of the tracts that we are interested in 
tracts = list(parking_dif.query("perc_dif > 10 | perc_dif < -10 ").index)

In [387]:
#converting to integers 
for i in range(len(tracts)):   
    tracts[i] = int(tracts[i])


In [388]:
tracts

[4301, 5302, 7200, 8200, 9200, 9300, 23801, 81100, 90102, 90900]

In [391]:
park_dif_block_merged = pd.merge(data_2013[data_2013.Tract== tracts[-1]], data_2018[data_2018.Tract== tracts[-1]], how='outer', on='Block').dropna()
park_dif_block_tableau = park_dif_block_merged[['Tract_x','Block', 'Total Stalls_x', 'Total Stalls_y' ]]
park_dif_block_tableau = park_dif_block_tableau.rename(columns={"Total Stalls_x": "Total Stalls 2013","Total Stalls_y": "Total Stalls 2018"})
park_dif_block_tableau['abs_dif'] = park_dif_block_tableau['Total Stalls 2018'] - park_dif_block_tableau['Total Stalls 2013']
park_dif_block_tableau['perc_dif'] = (park_dif_block_tableau['Total Stalls 2018'] - park_dif_block_tableau['Total Stalls 2013'])/park_dif_block_tableau['Total Stalls 2013']*100

In [392]:
for i in range(len(tracts)-1):
    park_dif_block_merged = pd.merge(data_2013[data_2013.Tract== tracts[i]], data_2018[data_2018.Tract== tracts[i]], how='outer', on='Block').dropna()
    park_dif_block = park_dif_block_merged[['Tract_x','Block', 'Total Stalls_x', 'Total Stalls_y' ]]
    park_dif_block = park_dif_block.rename(columns={"Total Stalls_x": "Total Stalls 2013","Total Stalls_y": "Total Stalls 2018"})
    park_dif_block['abs_dif'] = park_dif_block['Total Stalls 2018'] - park_dif_block['Total Stalls 2013']
    park_dif_block['perc_dif'] = (park_dif_block['Total Stalls 2018'] - park_dif_block['Total Stalls 2013'])/park_dif_block['Total Stalls 2013']*100
    park_dif_block_tableau = park_dif_block_tableau.append(park_dif_block)

In [402]:
park_dif_block_tableau[park_dif_block_tableau.Tract_x == '023801']

,Tract_x,Block,Total Stalls 2013,Total Stalls 2018,abs_dif,perc_dif
0,023801,1013,210.0,155.0,-55.0,-26.190476
1,023801,1002,917.0,288.0,-629.0,-68.593239
3,023801,1004,20.0,20.0,0.0,0.000000
4,023801,1005,93.0,93.0,0.0,0.000000


In [396]:
## Add leading zeros to the integer column in Python
 
park_dif_block_tableau['Tract_x']=park_dif_block_tableau['Tract_x'].astype(str).apply(lambda x: '{0:0>6}'.format(x))

#another way to add leading zeros 
#park_dif_block_tableau['Tract_x'].apply(lambda x: '{0:0>6}'.format(x))


In [ ]:
park_dif_block_tableau.to_excel("parking_dif.xlsx")